In [1]:
import os
from glob import glob
import pandas as pd
import exifread
import time 
import warnings
warnings.filterwarnings("ignore")
# https://in-the-sky.org/sunrise.php?startday=31&startmonth=5&startyear=2022&interval=4&tz=0

In [2]:
class PhotosFromTheEvening():
    
    def __init__(self,folder, sunset):
        self.folder_name = folder
        self.sunset_name = sunset
        self.df_images = self.getDataSet()
        self.df_sunset = self.get_file()
        df_results = self.df_images['image_nane'].apply(self.getPhotosFromTheEvening)
    
    # Convert the data to a datetime format
    toDatetime = lambda data: pd.to_datetime(data,infer_datetime_format=True)
    
    def getDataSet(self):
        # Load image from a folder named "input_data"
        list_of_images = glob(self.folder_name + "/*.jpg")
        # Turn a list into a data frame
        df_images = pd.DataFrame({'image_nane':list_of_images})
        return df_images
    
    def get_file(self):
        # Load csv from a file named "sunrise_sunset_times_jerusalem.csv"
        df_sunset=pd.read_csv(self.sunset_name)
        # Convert the columns 'Month' and 'Year' to int
        df_sunset[['Month','Year']] = df_sunset[['Month','Year']].astype(int)
        return df_sunset
    
    def getTimeSunset(self, date_obj):
        '''
        A method that receives the image data (date and time of shooting) checks the sunset time of the sun 
        according to the image data. Returns the number of minutes and hour of sunset.
        '''
        # Check if the year and month are the same as those obtained from the image, then select the data 
        data=self.df_sunset.loc[ (self.df_sunset['Year']== date_obj.year) & (self.df_sunset['Month']==date_obj.month) ]
        # From the data, select the sunset time
        data['Sunset_time'] = self.__class__.toDatetime(data['Set'])
        # Convert the hours and minutes of sunset to int
        minute_sunset, hour_sunset = int(data['Sunset_time'].dt.minute), int(data['Sunset_time'].dt.hour)
        return minute_sunset, hour_sunset 
             
            
    def getPhotosFromTheEvening(self, img):
        '''
        A method that receives the image. Finds data from the image, compares with the sunset data of the sun.
        Returns the name of the  images that was taken in the evening.
        '''
        # Open up the file for reading
        exif = exifread.process_file(open(img, 'rb'))
        # If there is metadata for the image
        if exif:
            date_obj = self.__class__.toDatetime(str(exif['EXIF DateTimeOriginal']))
            #geo = {i:exif[i] for i in exif.keys() if i.startswith('Image GPS')}
            #print(geo)
            minute_sunset, hour_sunset =self.getTimeSunset(date_obj)
            # Check whether the photo was taken during the evening. (After sunset and befor 24)
            if date_obj.hour >= hour_sunset and date_obj.minute>= minute_sunset and date_obj.hour <= 24:
                print(img.split("\\")[-1])


In [3]:
if __name__ == "__main__":
    # Check If Path Exists
    try:
        folder, sunset = "input_data", 'sunrise_sunset_times_jerusalem.csv'
        my_file_handle=os.path.exists(folder)
        PhotosFromTheEvening(folder, sunset)
    except IOError:
        print("File not found or path is incorrect")

pic1.jpg
pic4.jpg
